# GRU model

We start by importing the necessary libraries

In [ ]:
import torch
import matplotlib.pyplot as plt
from torch import nn
from torch.utils.data import DataLoader
from core.util.plot_predictions import plot_predictions
from core.util.save_model import save_model, load_parameters
from core.util.get_datasets import cross_validation
from core.util.trefor_dataset import TreforData
from core.models import GRU
from core.models.model_training import train_model, test_model
from core.util.hyperparameter_configuration import get_hyperparameter_configuration
from core.util.metrics import mae, rmse, smape

## Setup
This section contains the setup for converting our data to something PyTorch can understand.
### Set hyperparameters for the model

In [ ]:
experiment_name = "01_GRU_trefor_park"
features = {}
model_input_size = len(features) + 1

In [ ]:
experiment_parameters = load_parameters(experiment_name)
learning_rate = experiment_parameters["learning_rate"]
batch_size = experiment_parameters["batch_size"]
lookback = experiment_parameters["lookback"]
num_layers = experiment_parameters["num_layers"]
# torch.manual_seed(experiment_parameters["seed"])

experiment_parameters

In [ ]:
hyperparameters = get_hyperparameter_configuration()
hidden_size = hyperparameters["hidden_size"]
epochs = hyperparameters["epochs"]
horizon = hyperparameters["horizon"]
loss_function = hyperparameters["loss_function"]
dropout_rate = hyperparameters["dropout_rate"]
train_days = hyperparameters["train_days"]
val_days = hyperparameters["val_days"]
test_days = hyperparameters["test_days"]
early_stopper = hyperparameters["early_stopper"]

hyperparameters

In [ ]:
features = {
    "Month_x": True,
    "Month_y": True,
    "Hour_x": True,
    "Hour_y": True,
    "Day_x": True,
    "Day_y": True,
}

### If the host has CUDA, it will use the GPU for computation

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

### Creation of our simple GRU model
The implementation consist of three layers, defined in the `forward` method.
1. GRU
2. LeakyReLU
3. Linear

In [ ]:
def get_model() -> nn.Module:
    """Get the model for training folds."""
    model = GRU(
        input_size=model_input_size,
        hidden_size=hidden_size,
        num_layers=num_layers,
        dropout_rate=dropout_rate,
        horizon=horizon,
        lookback=lookback,
    )
    model.to(device)
    return model

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test = cross_validation(
    lookback=lookback,
    horizon=horizon,
    train_days=train_days,
    val_days=val_days,
    test_days=test_days,
    features=features,
)

## Main loop
Iterate all the cross validation folds to find the best one.

For each of these, we do the full iteration of epochs with training and validation.
Lastly, we run the test set on the given fold to see how it performs.

In [ ]:
# convert to dataset that can use dataloaders
train_dataset = TreforData(x_train, y_train, device)
val_dataset = TreforData(x_val, y_val, device)
test_dataset = TreforData(x_test, y_test, device)

# initialize the dataloaders, without shuffeling the data between epochs
training_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
validation_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
testing_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
for i in range(3):
    early_stopper.reset()
    experiment_iteration_name = "00_LSTM_trefor_park_iteration_" + str(i)
    best_train_loss, best_val_loss, best_model = train_model(
        model=get_model(),
        training_loader=training_loader,
        validation_loader=validation_loader,
        learning_rate=learning_rate,
        early_stopper=early_stopper,
    )

    t_loss, predicted = test_model(
        best_model=best_model,
        loss_function=loss_function,
        testing_loader=testing_loader,
    )

    model_mae = mae(y_test.tolist(), predicted)
    model_rmse = rmse(y_test.tolist(), predicted)
    model_smape = smape(y_test.tolist(), predicted)

    print(experiment_name)
    print(f"MAE: {model_mae}")
    print(f"RMSE: {model_rmse}")
    print(f"SMAPE: {model_smape}")

    save_model(
        model=best_model,
        model_name=experiment_name,
        train_loss=best_train_loss,
        val_loss=best_val_loss,
        mae=model_mae,
        rmse=model_rmse,
        smape=model_smape,
        overwrite=True,
    )

## Plotting
### Training- and validation loss

In [ ]:
plt.clf()
plt.plot(best_train_loss, label="Training Loss")
plt.plot(best_val_loss, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

### Predictions- and actual consumption

In [ ]:
plot_predictions(110, 124, y_test, predicted)

Plot predictions for the first 7 days, last 7, and a week in the middle of the test set

In [ ]:
plot_predictions(0, 6, y_test, predicted)

Plot predictions for the last 7 days in the test set

In [ ]:
plot_predictions(-7, -1, y_test, predicted)

Plot predictions for 7 days in the in the middle of the test set.

In [ ]:
days_count = len(y_test) / 24
plot_predictions(int(days_count / 2), int(days_count / 2 + 7), y_test, predicted)